## tfrecファイルの作成＋datasetへの登録 概要
- 前提
  - /kaggle/working直下は20GBの制限があるため、ここには作成しない。(Save方法したいで永続化される
  - /kaggle/tmp 等の自作したディレクトリは100GBまで作成できそうなので、そこに作成する。(この中のファイルはコードのセッションが終了すると削除される。)
  - datasetへの登録はKaggle APIを使用
  - トリミングの情報は拝借してきた  

- 手順
1. データセット一時保存用ディレクトリを作成
   - /kaggle/tmp/happy-whale-and-dolphin-dataset

2. データセット一時保存用ディレクトリに、以下のCSVファイルを作成
   - train_label_box.csv  
     訓練画像用CSV(train.csv)について、正解ラベルのエンコード、トリミング情報を付与したファイル
   - test_label_box.csv  
     評価画像について、トリミング情報を付与したファイル
   - label_master.csv  
     正解ラベル、エンコードした数値の対応マスタCSV  

3. 訓練画像をtfrec化してデータセット一時保存用ディレクトリに作成
   - tfrec内の1画像は以下の情報を保存
     - image:画像の行列
     - image_name:画像の名称　※学習には使わない
     - target:正解ラベル(エンコード後)
     - species:イルカ・クジラの種類(エンコード後) ※学習には使わない
     - box:トリミングの枠
   - ファイル名  
     train-images-{SEQ3桁}.tfrec
   - tfrecは10ファイル分作成(1ファイル約5104枚の画像)
     
4. テスト画像をtfrec化して、データセット一時保存用ディレクトリに作成
   - ファイル名  
     test-images.tfrec
   - target,speciesは情報がいため-1固定で作成

5. Kaggle APIを使用してデータセット一時保存用ディレクトリに作成したデータ一式のdatasetを作成
   - Kaggle APIのドキュメント  
     https://github.com/Kaggle/kaggle-api

In [ ]:
from math import ceil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

%config Completer.use_jedi = False

In [ ]:
# ライブラリのバージョンチェック
print(np.__version__)
print(pd.__version__)
print(tf.__version__)

In [ ]:
# 1. データセット一時保存用ディレクトリを作成
import os
import shutil
dataset_path = '/kaggle/tmp/happy-whale-and-dolphin-dataset'
if os.path.exists(dataset_path):
    shutil.rmtree(dataset_path)
os.makedirs('/kaggle/tmp/happy-whale-and-dolphin-dataset', exist_ok=True)

In [ ]:
# オリジナルデータセットの保存パスを定義
INPUT_BASE = '../input/'
INPUT_DATA_PATH = f'{INPUT_BASE}happy-whale-and-dolphin/'
TRAIN_IMAGES_PATH = f'{INPUT_DATA_PATH}train_images/'
TEST_IMAGES_PATH = f'{INPUT_DATA_PATH}test_images/'

# トリミング用のboxが格納されたデータセットの保存パスを定義
CROPPED_DATA_PATH = f'{INPUT_BASE}cropped-dataset/'

# 出力用のデータセット
#OUTPUT_DIR_PATH = '/kaggle/working/'
OUTPUT_DIR_PATH = '/kaggle/tmp/happy-whale-and-dolphin-dataset/'
OUTPUT_TRAIN_IMAGES_PATH = f'{OUTPUT_DIR_PATH}train_images/'
OUTPUT_TEST_IMAGES_PATH = f'{OUTPUT_DIR_PATH}test_images/'

# 出力用のファイル
#COPY_SAMPLE_CSV = 'train_sample.csv'
TRAIN_LABEL_CSV = 'train_label_box.csv'
TEST_LABEL_CSV = 'test_label_box.csv'
LABEL_MASTER_CSV = 'label_master.csv'

TRAIN_IMAGES_TFREC = 'train-images-{}-{}.tfrec'
TEST_IMAGES_TFREC = 'test-images-{}-{}.tfrec'

# 作業に使用する画像数を定義
IMAGE_COUNT = 10

In [ ]:
# 2. train_label_box.csvの作成
# 正解ラベルをエンコーディングしたデータが記載された訓練データCSVを出力
def write_label_encode_train_csv():
    # 正解ラベルのエンコーディング(説明変数として利用するわけではないため、ラベルエンコーディングで実施)
    df_train = pd.read_csv(f'{INPUT_DATA_PATH}train.csv')
    le = LabelEncoder()

    # individual_idのエンコード
    encoded_individual_id = pd.Series(le.fit_transform(df_train['individual_id']))
    df_train['encoded_individual_id'] = encoded_individual_id

    # speciesのエンコード
    encoded_species = pd.Series(le.fit_transform(df_train['species']))
    df_train['encoded_species'] = encoded_species
    
    # トリミング用のbox情報をセットを読み込んで、訓練データCSVに追加
    df_train_cropped_info = pd.read_csv(f'{CROPPED_DATA_PATH}train-cropped-dataset.csv')
    df_train['box'] = df_train_cropped_info['box']
    
    # エンコードした正解ラベルが記載されたCSVファイルを出力
    df_train.to_csv(f'{OUTPUT_DIR_PATH}{TRAIN_LABEL_CSV}')

write_label_encode_train_csv()

In [ ]:
# 2. test_label_box.csv の作成
# テストデータのトリミングしたbox情報を格納したcsvを出力
def write_test_box_csv():
    # 提出用サンプルにテスト画像が登録されているため、これをベースとする。
    df_test = pd.read_csv(f'{INPUT_DATA_PATH}sample_submission.csv')
    # "predictions"項目を削除する。
    df_test = df_test.drop('predictions', axis=1)

    # トリミング用のbox情報をセットを読み込んで、訓練データCSVに追加
    df_test_cropped_info = pd.read_csv(f'{CROPPED_DATA_PATH}test-cropped-dataset.csv')
    df_test['box'] = df_test_cropped_info['box']
    
    print(df_test.head())
    
    # エンコードした正解ラベルが記載されたCSVファイルを出力
    df_test.to_csv(f'{OUTPUT_DIR_PATH}{TEST_LABEL_CSV}')

write_test_box_csv()

In [ ]:
# 2. label_master.csv の作成
# (正解ラベルのマスタcsvを作成)
def create_label_master():
    # 正解ラベルのエンコーディング(説明変数として利用するわけではないため、ラベルエンコーディングで実施)
    df_train = pd.read_csv(f'{OUTPUT_DIR_PATH}{TRAIN_LABEL_CSV}')
    df_train = df_train.groupby(['encoded_individual_id', 'individual_id'],as_index=False)
    df_train = df_train.size()
    
    # "size"項目を削除する。
    df_train = df_train.drop('size', axis=1)
    df_train = df_train.rename(columns={'encoded_individual_id': 'index'})
    df_train.to_csv(f'{OUTPUT_DIR_PATH}{LABEL_MASTER_CSV}', index=False)
    
create_label_master()

In [ ]:
# # https://www.kaggle.com/lextoumbourou/happywhale-tfrecords-with-bounding-boxes
# def read_bbox(bbox):
#     return np.array([int(i) for i in bbox.split()])

# tfrecの作成関数類たち
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def _float_feature(value):
#   """Returns a float_list from a float / double."""
#   return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bb_feature(bb):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=bb))

#def serialize_example(image,image_name,target,species,yolov5_bb,detic_bb):
# 画像のシリアライズ(イメージ、正解ラベルを付与したデータセットを作成)
def serialize_image(image, image_name, target, species, box):
    feature = {
        'image': _bytes_feature(image),
        'image_name': _bytes_feature(image_name),
        'target': _int64_feature(target),
        'species': _int64_feature(species),
        'box': _bb_feature(box)
        #'yolov5_box': _bb_feature(yolov5_bb),
        #'detic_box': _bb_feature(detic_bb)
    }
    serialize_data = tf.train.Example(features=tf.train.Features(feature=feature))
    return serialize_data.SerializeToString()

In [ ]:
# 訓練画像の総枚数
IMAGE_TOTAL_COUNT = len(pd.read_csv(f'{OUTPUT_DIR_PATH}{TRAIN_LABEL_CSV}'))
# 訓練画像の1TFRECに保存する画像数
IMAGE_FOLD = ceil(IMAGE_TOTAL_COUNT / 10)
print(IMAGE_FOLD)

# テスト画像の総枚数
TEST_TOTAL_COUNT = len(pd.read_csv(f'{OUTPUT_DIR_PATH}{TEST_LABEL_CSV}'))
TEST_IMAGE_FOLD = ceil(TEST_TOTAL_COUNT / 10)
print(TEST_IMAGE_FOLD)
print(TEST_TOTAL_COUNT)

In [ ]:
# 4. テスト画像をtfrec化して、データセット一時保存用ディレクトリに作成
from math import ceil

# 訓練画像をtfrecファイルに変換
def train_image_to_tfrec():
    tfr_filename = f'{OUTPUT_DIR_PATH}{TRAIN_IMAGES_TFREC}'
    train_df = pd.read_csv(f'{OUTPUT_DIR_PATH}{TRAIN_LABEL_CSV}')
    
    total_row_count = len(train_df)
    print(total_row_count)

    tfrec_index = 0
    remaining_row_count = total_row_count

    row_index = 0
    while True:
        # tfrecのindexを取得
        tfrec_index = tfrec_index + 1
        
        if tfrec_index >= 3:
            break

        # tfrecの画像数を取得
        if remaining_row_count >= IMAGE_FOLD:
            tfrec_data_count = IMAGE_FOLD
        else:
            tfrec_data_count = remaining_row_count

        # tfrec名称を取得
        tfrec_filename = tfr_filename.format(str(tfrec_index).zfill(3), str(tfrec_data_count))

        # 画像を作成
        with tf.io.TFRecordWriter(tfrec_filename) as writer:
            start = (tfrec_index - 1) * IMAGE_FOLD
            end = start + tfrec_data_count

            for row_index in range(start, end):
                row = train_df.iloc[row_index]

                # 画像ファイル名
                image_id = row['image']
                # 正解ラベル(エンコーディング済み)
                target = row['encoded_individual_id']
                # イルカ・クジラの分類(エンコーディング済み)
                species = row['encoded_species']

                # 画像の読み込み
                image_path = f"{TRAIN_IMAGES_PATH}{image_id}"
                image_encoded = tf.io.read_file(image_path)
                image_name = str.encode(image_id)

                # トリミング領域の取得
                if type(row['box']) is float:
                    box = [-1, -1, -1, -1]
                else:
                    #box = list(read_bbox(row['box']))
                    box = [int(b) for b in row['box'].split()]

                serialize_image_data = serialize_image(image_encoded, image_name, target, species, box)
                writer.write(serialize_image_data)
            
            print(f'{tfrec_filename} 作成 ここまでの画像数：{row_index + 1}')

        # 残りの画像件数を取得
        remaining_row_count = remaining_row_count - IMAGE_FOLD
        # 全ての画像を処理しきったらループを抜ける
        if remaining_row_count <= 0:
            break
        
    print(f'index:{str(tfrec_index)}')

train_image_to_tfrec()

In [ ]:
# # 4. テスト画像をtfrec化して、データセット一時保存用ディレクトリに作成
# from math import ceil

# # 訓練画像をtfrecファイルに変換
# def train_image_to_tfrec():
#     tfr_filename = f'{OUTPUT_DIR_PATH}{TRAIN_IMAGES_TFREC}'
#     train_df = pd.read_csv(f'{OUTPUT_DIR_PATH}{TRAIN_LABEL_CSV}')
    
#     total_row_count = len(train_df)
#     print(total_row_count)
#     tfrec_count = ceil(total_row_count / IMAGE_FOLD) 
    
#     for i in range(1, tfrec_count + 1):
#         tfrec_filename = tfr_filename.format(str(i).zfill(3))

#         with tf.io.TFRecordWriter(tfrec_filename) as writer:
#             for index in range(0, IMAGE_FOLD):
#                 row_index = index + ((i - 1) * IMAGE_FOLD)
#                 if row_index >= total_row_count:
#                     break

#                 row = train_df.iloc[row_index]

#                 # 画像ファイル名
#                 image_id = row['image']
#                 # 正解ラベル(エンコーディング済み)
#                 target = row['encoded_individual_id']
#                 # イルカ・クジラの分類(エンコーディング済み)
#                 species = row['encoded_species']

#                 # 画像の読み込み
#                 image_path = f"{TRAIN_IMAGES_PATH}{image_id}"
#                 image_encoded = tf.io.read_file(image_path)
#                 image_name = str.encode(image_id)

#                 # トリミング領域の取得
#                 if type(row['box']) is float:
#                     box = [-1, -1, -1, -1]
#                 else:
#                     #box = list(read_bbox(row['box']))
#                     box = [int(b) for b in row['box'].split()]

#                 serialize_image_data = serialize_image(image_encoded, image_name, target, species, box)
#                 writer.write(serialize_image_data)
            
#             print(f'{tfrec_filename} 作成 ここまでの画像数：{row_index}')

#     print(f'画像数:{row_index}')

# train_image_to_tfrec()

In [ ]:
# 4. テスト画像をtfrec化して、データセット一時保存用ディレクトリに作成
def test_image_to_tfrec():
    tfr_filename = f'{OUTPUT_DIR_PATH}{TEST_IMAGES_TFREC}'
    test_df = pd.read_csv(f'{OUTPUT_DIR_PATH}{TEST_LABEL_CSV}')

    total_row_count = len(test_df)
    print(total_row_count)

    tfrec_index = 0
    remaining_row_count = total_row_count
    
    row_index = 0
    while True:
        # tfrecのindexを取得
        tfrec_index = tfrec_index + 1

        if tfrec_index >= 3:
            break
        
        # tfrecの画像数を取得
        if remaining_row_count >= TEST_IMAGE_FOLD:
            tfrec_data_count = TEST_IMAGE_FOLD
        else:
            tfrec_data_count = remaining_row_count

        # tfrec名称を取得
        tfrec_filename = tfr_filename.format(str(tfrec_index).zfill(3), str(tfrec_data_count))

        # 画像を作成
        with tf.io.TFRecordWriter(tfrec_filename) as writer:
            start = (tfrec_index - 1) * TEST_IMAGE_FOLD
            end = start + tfrec_data_count

            for row_index in range(start, end):
                row = test_df.iloc[row_index]

                # 画像ファイル名
                image_id = row['image']
                # 正解ラベル(正解が分からないので固定値)
                target = -1
                # イルカ・クジラの分類(正解が分からないので固定値)
                species = -1

                # 画像の読み込み
                image_path = f"{TEST_IMAGES_PATH}{image_id}"
                image_encoded = tf.io.read_file(image_path)
                image_name = str.encode(image_id)

                if type(row['box']) is float:
                    box = [-1, -1, -1, -1]
                else:
                    box = [int(b) for b in row['box'].split()]

                serialize_image_data = serialize_image(image_encoded, image_name, target, species, box)
                writer.write(serialize_image_data)
            
            print(f'{tfrec_filename} 作成 ここまでの画像数：{row_index + 1}')

        # 残りの画像件数を取得
        remaining_row_count = remaining_row_count - TEST_IMAGE_FOLD
        # 全ての画像を処理しきったらループを抜ける
        if remaining_row_count <= 0:
            break
        
    print(f'index:{str(tfrec_index)}')

test_image_to_tfrec()

In [ ]:
# # 4. テスト画像をtfrec化して、データセット一時保存用ディレクトリに作成
# def test_image_to_tfrec():
#     tfr_filename = f'{OUTPUT_DIR_PATH}{TEST_IMAGES_TFREC}'
#     test_df = pd.read_csv(f'{OUTPUT_DIR_PATH}{TEST_LABEL_CSV}')

#     test_total_count = len(test_df)
#     #tfrec_count = ceil(len(test_df) / IMAGE_FOLD)
#     #print(tfrec_count)
    
#     with tf.io.TFRecordWriter(tfr_filename) as writer:
#         for index, row in test_df.iterrows():
#             image_id = row['image']
#             # 正解ラベル(正解が分からないので固定値)
#             target = -1
#             # イルカ・クジラの分類(正解が分からないので固定値)
#             species = -1
            
#             # 画像の読み込み
#             image_path = f"{TEST_IMAGES_PATH}{image_id}"
#             image_encoded = tf.io.read_file(image_path)
#             image_name = str.encode(image_id)
            
#             if type(row['box']) is float:
#                 box = [-1, -1, -1, -1]
#             else:
#                 #box = list(read_bbox(row['box']))
#                 box = [int(b) for b in row['box'].split()]

#             serialize_image_data = serialize_image(image_encoded, image_name, target, species, box)
#             writer.write(serialize_image_data)
            
#             if index % 2000 == 0:
#                 print(f'テスト画像:{index}枚 作成済')

#     print(f'テスト画像tfrec作成済み')

# test_image_to_tfrec()

In [ ]:
# 5. Kaggle APIを使用してデータセット一時保存用ディレクトリに作成したデータ一式のdatasetを作成
## kaggle.jsonを作成する。これがルートディレクトリにないとKaggle APIが使用できない。
## このjsonはユーザアカウント画面のCreat New API Tokenから取得できる。
## ただし、直接コードにアップロードする方法が分からなかったので、内容をLunuxコマンドで直接書き込んだ。(usernameやkeyは自分のkaggle.jsonのものに読み替えて下さい)
!echo '{"username":"npc0302","key":"0de12965922b9485b11174c575648272"}' > /root/.kaggle/kaggle.json
## 600にするとkaggle.jsonを他ユーザに公開しない。
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# 5. Kaggle APIを使用してデータセット一時保存用ディレクトリに作成したデータ一式のdatasetを作成
## LinuxコマンドでKaggle APIをたたいてdatasetに登録するAPIを実行(データセットはここではnpx0302ユーザ配下のtest01　としている。)
## 以下はdatasetを作成するために必要なmetadataのjsonファイルを作成するコマンド。
## 詳細は以下を参照
##   https://github.com/Kaggle/kaggle-api#datasets
!kaggle datasets init -p /kaggle/tmp/happy-whale-and-dolphin-dataset
!cat /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata.json | jq '.title|="happy-whale-and-dolphin-tfrec"' > /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata-1.json
!cat /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata-1.json | jq '.id|="npc0302/happy-whale-and-dolphin-tfrec"' > /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata-2.json
!mv -f /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata-2.json /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata.json
!cat /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata.json
!rm /kaggle/tmp/happy-whale-and-dolphin-dataset/dataset-metadata-1.json

## 一時作業フォルダ/kaggle/tmp/happy-whale-and-dolphin-datasetに存在するファイル一式をdatasetにアップロードする。
## (--dir-mode zipは、この中にあるサブディレクトリをzip化してアップロードするコマンドだが、重くて途中で落ちたのでやめた。全ファイルを直下に配置する形式とした)
## (--dir-mode zipがない場合、内部のサブディレクトリは無視されるっぽい)
#!kaggle datasets create -p /kaggle/tmp/happy-whale-and-dolphin-dataset --dir-mode zip
!kaggle datasets create -p /kaggle/tmp/happy-whale-and-dolphin-dataset